In [1]:
import pandas as pd

In [2]:
dataset = pd.read_csv('toyota.csv')
# re-organizing columns
dataset = dataset[dataset.columns[[0,1,3,4,5,6,7,8,2]]]
dataset.head()

,model,year,transmission,mileage,fuelType,tax,mpg,engineSize,price
0,GT86,2016,Manual,24089,Petrol,265,36.2,2.0,16000
1,GT86,2017,Manual,18615,Petrol,145,36.2,2.0,15995
2,GT86,2015,Manual,27469,Petrol,265,36.2,2.0,13998
3,GT86,2017,Manual,14736,Petrol,150,36.2,2.0,18998
4,GT86,2017,Manual,36284,Petrol,145,36.2,2.0,17498


<h3><center>1. Check for NaN values</center></h3>

In [3]:
print(dataset.isnull().sum())

model           0
year            0
transmission    0
mileage         0
fuelType        0
tax             0
mpg             0
engineSize      0
price           0
dtype: int64


In [4]:
dataset.shape

(6738, 9)

<h3><center>2. Douplicate values</center></h3>

In [5]:
# searching for douplicate values
dataset.groupby(dataset.columns.tolist(),as_index=False).size().sort_values()

model   year  transmission  mileage  fuelType  tax  mpg   engineSize  price
 Auris  2007  Automatic     84000    Petrol    205  40.9  1.6         3395     1
 Verso  2011  Manual        67085    Petrol    200  41.5  1.6         5999     1
                            54000    Diesel    145  53.3  2.0         5995     1
                            47570    Petrol    200  41.5  1.6         6795     1
                            29761    Diesel    145  53.3  2.0         7995     1
                                                                              ..
 Aygo   2016  Manual        12935    Petrol    0    69.0  1.0         8450     2
        2019  Manual        2000     Petrol    145  57.7  1.0         10350    3
                            1500     Petrol    145  56.5  1.0         9999     3
                            25       Petrol    145  56.5  1.0         9995     3
 RAV4   2015  Manual        45757    Diesel    125  57.6  2.0         13500    3
Length: 6699, dtype: int64

In [6]:
dataset = dataset.drop_duplicates()
dataset.shape

(6699, 9)

In [7]:
dataset.groupby(dataset.columns.tolist(),as_index=False).size().sort_values()

model   year  transmission  mileage  fuelType  tax  mpg   engineSize  price
 Auris  2007  Automatic     84000    Petrol    205  40.9  1.6         3395     1
 Verso  2011  Manual        67085    Petrol    200  41.5  1.6         5999     1
                            54000    Diesel    145  53.3  2.0         5995     1
                            47570    Petrol    200  41.5  1.6         6795     1
                            29761    Diesel    145  53.3  2.0         7995     1
                                                                              ..
 Aygo   2018  Manual        14716    Petrol    145  56.5  1.0         8222     1
                            14696    Petrol    145  68.9  1.0         7995     1
                            14680    Petrol    145  69.0  1.0         6995     1
                            12987    Petrol    145  56.5  1.0         8295     1
 Yaris  2020  Manual        4895     Petrol    150  49.6  1.0         12495    1
Length: 6699, dtype: int64

<h3><center>3. Unusual values  [zeros] </center></h3>

In [8]:
# we notice that min value of engineSize = 0 which is unusual!
dataset.describe()

,year,mileage,tax,mpg,engineSize,price
count,6699.000000,6699.000000,6699.000000,6699.000000,6699.000000,6699.000000
mean,2016.742797,22889.588745,94.549933,63.078728,1.471996,12529.799074
std,2.205271,19109.288501,73.942649,15.861037,0.435624,6358.562625
min,1998.000000,2.000000,0.000000,2.800000,0.000000,850.000000
25%,2016.000000,9488.500000,0.000000,55.400000,1.000000,8266.500000
50%,2017.000000,18571.000000,135.000000,62.800000,1.500000,10798.000000
75%,2018.000000,31059.500000,145.000000,70.600000,1.800000,14995.000000
max,2020.000000,174419.000000,565.000000,235.000000,4.500000,59995.000000


In [9]:
# check 0s of engine size => 6 rows
dataset['engineSize'][dataset.engineSize == 0].count()

6

In [10]:
# to solve this issue substitute with mean
dataset['engineSize'][dataset.engineSize == 0] = dataset.engineSize.mean()
dataset.describe()

,year,mileage,tax,mpg,engineSize,price
count,6699.000000,6699.000000,6699.000000,6699.000000,6699.000000,6699.000000
mean,2016.742797,22889.588745,94.549933,63.078728,1.473314,12529.799074
std,2.205271,19109.288501,73.942649,15.861037,0.433388,6358.562625
min,1998.000000,2.000000,0.000000,2.800000,1.000000,850.000000
25%,2016.000000,9488.500000,0.000000,55.400000,1.000000,8266.500000
50%,2017.000000,18571.000000,135.000000,62.800000,1.500000,10798.000000
75%,2018.000000,31059.500000,145.000000,70.600000,1.800000,14995.000000
max,2020.000000,174419.000000,565.000000,235.000000,4.500000,59995.000000


In [11]:
dataset.tax.mean()

94.5499328257949

In [12]:
# check 0s of tax 1787 hybrid + Petrol
dataset['tax'][dataset.tax == 0].count()

1787

In [13]:
# Note: based on UK road regulations, tax=0 means no emission so we accept that for hybrid but NOT for Petrol type
# Now exclude only petrol type with tax 0 from data, to see how many first:
dataset[(dataset.tax == 0) & (dataset.fuelType=='Petrol')]

,model,year,transmission,mileage,fuelType,tax,mpg,engineSize,price
857,Yaris,2016,Manual,24790,Petrol,0,66.0,1.0,7995
870,Yaris,2015,Manual,33000,Petrol,0,66.0,1.0,6995
895,Yaris,2016,Manual,8484,Petrol,0,66.0,1.0,8698
905,Yaris,2016,Manual,8246,Petrol,0,66.0,1.0,7298
929,Yaris,2016,Manual,17001,Petrol,0,66.0,1.0,7298
...,...,...,...,...,...,...,...,...,...
6076,C-HR,2017,Automatic,19315,Petrol,0,74.3,1.8,18999
6727,IQ,2009,Manual,43000,Petrol,0,65.7,1.0,3495
6728,IQ,2009,Manual,74095,Petrol,0,65.7,1.0,2699
6730,IQ,2013,Manual,41142,Petrol,0,64.2,1.0,4849


In [14]:
# drop rows with 0 tax and fuelType = Petrol
dataset_wo_0tax = dataset[~((dataset.tax == 0) & (dataset.fuelType=='Petrol'))]
dataset_wo_0tax.head(3)

,model,year,transmission,mileage,fuelType,tax,mpg,engineSize,price
0,GT86,2016,Manual,24089,Petrol,265,36.2,2.0,16000
1,GT86,2017,Manual,18615,Petrol,145,36.2,2.0,15995
2,GT86,2015,Manual,27469,Petrol,265,36.2,2.0,13998


<h3><center>4. Detect Outlires </center></h3>

In [15]:
# df: dataset, col: feature
def detect_oulires(df, col):
    """ Here we caculate outlires based on IQR method and then, we set
    the threshold """
    """ IQR = Q3 {0.75} - Q1 {0.25} """
    Q1 = dataset[col].quantile(0.25)
    Q3 = dataset[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_boundary = Q1 - 1.5 * IQR
    upper_boundary = Q3 + 1.5 * IQR
    # return mask of T & F values    
    return  (df[col] < upper_boundary) & (df[col] > lower_boundary)


In [16]:
# now apply this function to all columns to remove outlires
cleaned_dataset = dataset[
                            detect_oulires(dataset, 'price') \
                          & detect_oulires(dataset, 'mpg')\
                          & detect_oulires(dataset, 'year')\
                          & detect_oulires(dataset, 'tax') \
                          & detect_oulires(dataset, 'engineSize')
                         ]

cleaned_dataset.head(3)

,model,year,transmission,mileage,fuelType,tax,mpg,engineSize,price
0,GT86,2016,Manual,24089,Petrol,265,36.2,2.0,16000
1,GT86,2017,Manual,18615,Petrol,145,36.2,2.0,15995
2,GT86,2015,Manual,27469,Petrol,265,36.2,2.0,13998


In [17]:
cleaned_dataset.describe()

,year,mileage,tax,mpg,engineSize,price
count,5928.000000,5928.000000,5928.000000,5928.000000,5928.000000,5928.000000
mean,2017.061572,20870.898111,91.851383,63.215671,1.424499,12022.247470
std,1.423046,15366.608234,70.601499,11.421091,0.391148,4787.205062
min,2014.000000,2.000000,0.000000,32.800000,1.000000,2991.000000
25%,2016.000000,9508.500000,0.000000,56.500000,1.000000,8483.750000
50%,2017.000000,17710.500000,135.000000,65.700000,1.500000,10795.000000
75%,2018.000000,28977.750000,145.000000,69.000000,1.800000,14263.500000
max,2020.000000,142000.000000,265.000000,86.000000,2.500000,25000.000000


In [18]:
# Export cleaned data to csv file for regression model
cleaned_dataset.to_csv('toyota_cleaned_dataset.csv', index = False,header=True)